In [1]:
# from surprise import BaselineOnly
# from surprise import NMF
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise.model_selection import KFold
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import AlgoBase


from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline

import os
# import sys
import math
import statistics
import collections

import sklearn as sk
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.decomposition import NMF
import numpy as np
import pandas as pd

import random
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

import datetime
from datetime import datetime
from time import time
import timeit

# from multiprocessing import Pool

In [2]:
class RatingDataset:
    import numpy as np
    from scipy import sparse
    
    def __init__(self):
        self.rating_mat = None
        # self.time_mat = None
        self._data_file_path = ''
        
        # list of raw user_IDs (dataset IDs)
        self.items = []
        self.users = []
        self.item_n = 0
        self.user_n = 0
        
        # maps raw user_id to user_iid(or inner id)
        self.user_to_iid = {}
        # maps user inner id to dataset raw ID
        self.user_to_ID = {}
        # maps raw item_id (dataset) to item_iid(or inner id)
        self.item_to_iid = {}
        # maps item inner id to dataset raw ID
        self.item_to_ID = {}
        
        # list of triples of (item, rating, timestamp) for each user_iid. 
        # TODO: In case there were no Timestamp in the data, pairs of (item, rating) will be kept
        self.user_ratings = []
        # list of pair of (user, rating) for each item_iid
        self.item_ratings = []
        
        
    def __get_line_format_indices(self, line_format):
        # specifying the order of 'user, item, rating, timestamp' in each line 
        lf_sp = line_format.split(' ')
        # if len(lf_sp) != 4:
        #     raise Exception('''Bad line format!
        #     line_format should be space-separated and it should always specified by 
        #     "user item rating timestamp" with any order!''')
        user_idx = -1
        item_idx = -1
        rating_idx = -1
        # timestamp_idx = -1
        for c in range(len(lf_sp)):
            if lf_sp[c] == 'user':
                user_idx = c
            elif lf_sp[c] == 'item':
                item_idx = c
            elif lf_sp[c] == 'rating':
                rating_idx = c
            # elif lf_sp[c] == 'timestamp':
            #     timestamp_idx = c
            else:
                raise Exception('line_format must be exactly dictated by one of: (user/item/rating/timestamp) separated by sep!')
        
        # return user_idx, item_idx, rating_idx, timestamp_idx
        return user_idx, item_idx, rating_idx
    
    
    '''
        Read the rating data from file and parse it and then make the dataset.
    '''
    # def read_from_file(self, data_fn, skip_lines=0, sep=',', line_format='user item rating timestamp'):
    def read_from_file(self, data_fn, skip_lines=0, sep=',', line_format='user item rating'):
        
        # user_fmt_idx, item_fmt_idx, rating_fmt_idx, timestamp_fmt_idx = self.__get_line_format_indices(line_format)
        user_fmt_idx, item_fmt_idx, rating_fmt_idx = self.__get_line_format_indices(line_format)
        
        file = open(data_fn, 'r')
        
        # skip lines that are specified from input
        for _ in range(skip_lines):
            file.readline()
            
        # users list as in input file
        users_lin = []
        items_lin = []
        ratings_lin = []
        # timestamps_lin = []
        for l in file:
            lsp = l.split(sep)
            user_id = lsp[user_fmt_idx]
            item_id = lsp[item_fmt_idx]
            rating = float(lsp[rating_fmt_idx])
            # timestamp = int(lsp[timestamp_fmt_idx].strip('\n'))
            
            users_lin.append(user_id)
            items_lin.append(item_id)
            ratings_lin.append(rating)
            # timestamps_lin.append(timestamp)
            
        self.users = list(set(users_lin))
        self.items = list(set(items_lin))
        
        self.user_n = len(self.users)
        self.item_n = len(self.items)
        
        '''note that raw ids are in STRING format, and the iid in INTEGER format!'''
        # set the mappings
        for idx in range(self.user_n): 
            self.user_to_iid[self.users[idx]] = idx
            
        for idx in range(self.user_n):
            self.user_to_ID[idx] = self.users[idx] 
            
        for idx in range(self.item_n):
            self.item_to_iid[self.items[idx]] = idx 
            
        for idx in range(self.item_n):
            self.item_to_ID[idx] = self.items[idx] 
        
        # init rating matrix
        self.rating_mat = sparse.lil_matrix((self.user_n, self.item_n))
        # self.time_mat = sparse.lil_matrix((self.user_n, self.item_n))
        for idx in range(len(users_lin)):
            user_iid = self.user_to_iid[users_lin[idx]]
            item_iid = self.item_to_iid[items_lin[idx]]
            rating = ratings_lin[idx]
            self.rating_mat[user_iid, item_iid] = rating
            # self.time_mat[user_iid, item_iid] = timestamps_lin[idx]
            
            
    def list_users_ratings(self, rating_matrix):
        # finding the user and item ratings
        user_ratings = []
        for user_iid in range(self.user_n):
            # append a list for this user
            user_ratings.append([])
            user_nonze = np.nonzero(rating_matrix[user_iid])
            for item_iid in user_nonze[1]:
                # add items and its rating into the last user added to the list
                user_ratings[-1].append((item_iid, rating_matrix[user_iid, item_iid]))
                if rating_matrix[user_iid, item_iid] == 0:
                    raise Exception('Found zero rating in nonzero ratings of user with inner id %d and item iid %d!' % (user_iid, item_iid))
        return user_ratings
    
            
    def list_items_ratings(self, rating_matrix):
        item_ratings = []
        for item_iid in range(self.item_n):
            # append a list for this item
            item_ratings.append([])
            item_nonze = np.nonzero(rating_matrix.T[item_iid])
            for user_iid in item_nonze[1]:
                # add users and its rating into the last item added to the list
                item_ratings[-1].append((user_iid, rating_matrix[user_iid, item_iid]))
                if rating_matrix[user_iid, item_iid] == 0:
                    raise Exception('Found zero rating in nonzero ratings of user with inner id %d and item iid %d!' % (user_iid, item_iid))
        return item_ratings
        
            
    def train_test_split(self, test_percent=0.2, least_userlen_test=10):
        if test_percent > 1:
            raise Exception('test_percent should be between 0 and 1.')
            
        user_ratings = self.list_users_ratings(self.rating_mat)
        
        mat = sparse.lil_matrix((self.user_n, self.item_n))
        user_tests = {}
        n_users_in_test = 0
        n_ratings_in_test = 0
        n_ratings_in_train = 0
        
        for user_iid in range(self.user_n):
            len_u = len(user_ratings[user_iid])
            if len_u >= least_userlen_test:
                n_users_in_test += 1
                test_len = int(len_u * test_percent)
                test_set_u = list(range(len_u))
#                 print(test_len, len_u)
                random.shuffle(test_set_u)
                
                train_set_u = test_set_u[test_len:][:]
                test_set_u = test_set_u[:test_len][:]
                
#                 print(len(train_set_u))
                
                for ir_idx in train_set_u:
                    # ir = the pair of (item, rating)
                    ir = user_ratings[user_iid][ir_idx]
                    mat[user_iid, ir[0]] = ir[1]
                    n_ratings_in_train += 1
                
                user_tests[user_iid] = []
                for ir_idx in test_set_u:
                    # ir = the pair of (item, rating)
                    ir = user_ratings[user_iid][ir_idx]
                    user_tests[user_iid].append(ir)
                    n_ratings_in_test += 1
                    
            else: # if no test set should be seprated from ratings of this user
                for ir in user_ratings[user_iid]:
                    # ir = the pair of (item, rating)
                    mat[user_iid, ir[0]] = ir[1]
                    n_ratings_in_train += 1
    
        print('\nNumber of users with some items in testset: %d' % n_users_in_test)
        print('Number of ratings in trainset: %d \t Number of ratings in testset: %d\n' % (n_ratings_in_train, n_ratings_in_test))
        return mat, user_tests

In [4]:
n_core = 45
dataset = RatingDataset()
data_fn = './filtered_ml_%icore.csv'%n_core
dataset.read_from_file(data_fn, skip_lines=1, line_format='user item rating', sep=',')

In [5]:
print('# users', dataset.user_n)
print('# items', dataset.item_n)


# user_tests is the test_mat
train_mat, test_mat = dataset.train_test_split(test_percent=0.2, least_userlen_test=10)


# ir = the pair of (item, rating)
# ir = user_ratings[user_iid][ir_idx]
user_ratings = dataset.list_users_ratings(dataset.rating_mat)
print(dataset.rating_mat.shape)

# users 551
# items 593

Number of users with some items in testset: 551
Number of ratings in trainset: 58307 	 Number of ratings in testset: 14293

(551, 593)


In [20]:
user_ratings = dataset.list_users_ratings(train_mat)
# user_ratings[0]
# train_df = pd.DataFrame(columns = ['item','rating'])


tr_lst = []

for user_iid in range(dataset.user_n): 
    # trainset or dataset.user_n?? we keep the users the same (user-fixed) so they are equal.
    
    if user_ratings[user_iid]:
        base_rec = pd.DataFrame(user_ratings[user_iid])
        base_rec[2] = user_iid

        tr_lst.append(base_rec[[2,0,1]])
    

train_df = pd.concat(tr_lst, ignore_index=True)
train_df.columns = ['user','item','rating']
train_df.head()

,user,item,rating
0,0,2,5.0
1,0,50,4.0
2,0,77,2.0
3,0,84,5.0
4,0,105,5.0


In [21]:
# test set to a dataframe
test_lst = []

for uiid in test_mat.keys():
    base_rec = pd.DataFrame(test_mat[uiid])
    base_rec[2] = uiid
    test_lst.append(base_rec[[2,0,1]])


test_df = pd.concat(test_lst, ignore_index=True)
test_df.columns = ['user','item','rating']
test_df.head()

,user,item,rating
0,0,365,4.0
1,0,174,5.0
2,0,528,4.0
3,0,515,2.0
4,0,522,4.0


- We had train_df

In [76]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# Pivot and create movie-user matrix
movie_user_mat = train_df.pivot(index='item', columns='user', values='rating').fillna(0)


# transform matrix to scipy sparse matrix
movie_user_mat_sparse = csr_matrix(movie_user_mat.values)


model_knn= NearestNeighbors(metric='cosine',algorithm='brute',n_neighbors=20, n_jobs=-1)

# fit - pass the sparse matrix to it
model_knn.fit(movie_user_mat_sparse)


# choose the # of recoms
n_recommendations = 10

# choose an item
idx = 100

#Making inference
distances, indices = model_knn.kneighbors(movie_user_mat_sparse[idx], n_neighbors=n_recommendations+1)

# get list of raw idx of recommendations
raw_recommends = sorted(list(zip(indices.squeeze().tolist(), 
                                 distances.squeeze().tolist())),
                        key=lambda x: x[1])[:0:-1]

for i, (idx, dist) in enumerate(raw_recommends):
        print('{0}: {1}, with distance of {2}'.format(i+1, idx, dist))

1: 364, with distance of 0.6958387436767748
2: 282, with distance of 0.6865957418915738
3: 21, with distance of 0.6824902685277933
4: 442, with distance of 0.6815396974484128
5: 139, with distance of 0.6812998359201858
6: 256, with distance of 0.6802311759405928
7: 115, with distance of 0.6766383520967707
8: 398, with distance of 0.672946263257415
9: 333, with distance of 0.6553548554773755
10: 37, with distance of 0.6417029522329425


In [ ]:
class NeighborBasedActiveLearner:

    def __init__(self, n_neighbors=10, metric='cosine', epochs=10):
        '''
        Neighbor-based active learner class.

        Args:
            n_neighbors:    int, number of neighbors to consider.
            metric:         str, metric used to calculate distance.
            epochs:         int, number of active learning steps.
        '''

        self.n_neighbors = n_neighbors
        self.epochs = epochs
        self.knn_model = NearestNeighbors(n_neighbors=self.n_neighbors, metric=metric)
        self.item_user_mat = None

    def fit(self, train_df):

        # initial fit, using 2%
        initial_train_df = train_df.sample(frac=0.02).reset_index(drop=True)

        self.item_user_mat = self._convert_df_to_mat(initial_train_df)
        self.knn_model.fit(self.item_user_mat)

        # active learning process
        for epoch in range(self.epochs):
            for user_iid in range(dataset.user_n):

                # get the items to be queried
                query_item_lst = self.query(user_iid, self.n_neighbors)
                candidate_df_u = train_df.loc[train_df['user'] == user_iid]
                query_df = candidate_df_u.loc[candidate_df_u['item'].isin(query_item_lst)].copy().reset_index(drop=True)
                # add to known matrix
                self.add_query(query_df)
        
            self.train()
        


    def _convert_df_to_mat(self, df):
        '''
        Convert DataFrame to sparse matrix
        
        Arg:
            df: DataFrame, training DataFrame
        
        Return:
            mat: scipy.sparse.csr_matrix, sparse matrix containing training data
        '''

        mat = sparse.csr_matrix((dataset.item_n, dataset.user_n))
        for _, row in df.iterrows():
            user_iid = int(row[0])
            item_iid = int(row[1])
            rating = row[2]
            mat[item_iid, user_iid] = rating
        
        return mat 

    def add_query(self, query_df):
        '''
        Add queried data to known matrix.
        
        Arg:
            query_df: DataFrame, new data to be added.
        '''

        for _, row in query_df.iterrows():
            user_iid = int(row[0])
            item_iid = int(row[1])
            rating = row[2]
            self.item_user_mat[item_iid, user_iid] = rating

    def train(self):
        '''
        Fit self.knn_model to known data.
        '''

        self.knn_model.fit(self.item_user_mat)

        # return self to enable learner.train().query()
        return self
    
    def query(self, user_iid, topk):
        '''
        Find the indices of candidates which should be queried for true ratings.

        Args:
            user_iid: int, inner id of the user to be queried
            topk:     int, number of queried candidates
        
        Return:
            top_items: numpy int array, top k items to be queried
        '''
        
        rated_before = np.nonzero(self.item_user_mat[:, user_iid])[0]
        
        if rated_before.size > 0:
            # the user has at least one rating elicited
            raw_recommends = {}
            for item_iid in rated_before:
                distances, indices = self.knn_model.kneighbors(self.item_user_mat[item_iid], n_neighbors=self.n_neighbors)
                sorted_pairs = sorted(list(zip(indices.squeeze().tolist(),
                                               distances.squeeze().tolist())),
                                      key=lambda x: x[1])[:0:-1]
                raw_recommends[item_iid] = sorted_pairs
            
            # get top 10 items
            top_items = []
            pos = 0
            while True:
                for item_iid in rated_before:
                    next_neighbor_iid = raw_recommends[item_iid][pos][0]
                    top_items.append(next_neighbor_iid)
                    if len(top_items) > topk - 1:
                        return np.array(top_items)
                
                pos += 1
                
        else:
            # the user has no rating in the known set
            top_items = random.sample(list(range(0, dataset.item_n)), topk)
            return np.array(top_items)

    def evaluate(self, test_mat):
        pass
